# Sistemas de ecuaciones no lineales

In [1]:
# Modulos
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets # Controles
from ipywidgets import interact, interact_manual

In [2]:
# Funciones auxiliares

## Derivada numerica
def derivada(f,x,tol = 1e-12, maxiter = 500):
    """Calcula la derivada de f en x (f'(x)) mediante diferencias progresivas.
    
    - Entrada >    
      f (function)  - Funcion de la cual se quiere conocer su derivada.
      x (float)     - Punto donde se quiere conocer el valor de f'.
      tol (float)   - Opcional. Tolerancia para la diferencia entre dos aproximaciones.
      maxiter (int) - Opcional. Numero maximo de iteraciones.
    
    - Salida >
      dfx (float) - Valor aproximado de f'(x).
    """
    d,h,it = np.inf,0.5,1
    dfx_a = f(x+1) - f(x)
    while abs(d) > tol and it < maxiter:
        dfx = (f(x+h) - f(x)) / h
        d = dfx - dfx_a
        dfx_a = dfx
        h /= 2
        it += 1
    return dfx

## Funciones de prueba
fun1 = lambda x:x**2-1.0
fun2 = lambda x:x**2-4.0*np.sin(x)

## Ceros de una función

Dada una función continua $f\colon[a,b]\subset\mathbb{R}\to \mathbb{R}$, se quiere encontrar un cero o raíz de $f$. Esto es, se quiere saber si existe $x_{0}\in[a,b]$ tal que $f(x_{0}) = 0$. Existen varios algoritmos para buscar ceros de funciones y a continuación se mostrarán algunos de ellos.

### Método de bisección

El algoritmo más simple para la búsqueda de raíces de una función continua $f$ en el intervalo $[a,b]$ es el método de bisección. Si existe una raíz en el intervalo $[a,b]$, el método consiste en ir dividiendo el intervalo a la mitad y seleccionar el subintervalo que contiene la raíz. Para garantizar la existencia de la raíz en el intervalo $[a,b]$ y en los subintervalos subsecuentes, el método se basa en el siguiente teorema

**Teorema** *Si $f$ es una función continua en el intervalo $[a,b]$ con $f(a)f(b) < 0$, entonces existe $c\in(a,b)$ tal que $f(c) = 0$.*

La idea es ir cambiando los extremos de los subintervalos para cumplir con las hipótesis del teorema y así garantizar que se seleccione el intervalo que contiene la raíz

#### Algoritmo (método de biscción)

* *Entrada* - $f$ función continua en $[a,b]$, $a$ y $b$ extremos del intervalo.
* *Salida*  - Raíz de $f$.

1. **Definir** $tol$.
2. **Mientras** $(b-a) > tol$ **hacer**    
    1. Calcular $$m = \dfrac{b+a}{2}$$
    2. **Si** $f(b)f(m)\leq 0$ **entonces**
        1. $a = m$
    3. **De lo contrario**
        1. $b = m$
3. **Regresar** $m$.

In [3]:
def biseccion(f,a,b,tol = 1e-8, maxiter = 500):
    """Busca un cero de la funcion f mediante el metodo de biseccion en el intervalo (a,b).
    - Entrada >
      f (function)    - Funcion de la cual se quiere conocer su raiz.
      a (float)       - Extremo inferior del intervalo.
      b (float)       - Extremo superior del intervalo.
      tol (float)     - Opcional. Tolerancia para la diferencia entre dos aproximaciones.
      maxiter (float) - Opcional. Numero maximo de iteraciones.
    
    - Salida >
      cero (float) - Valor aproximado de un cero .
      it (int)     - Número de iteraciones que le toma al metodo encontrar un cero.
    """
    it = 0
    while abs(b-a) > tol and it < maxiter:
        it += 1
        m = (b+a)/2
        if f(b)*f(m) <= 0:
            a = m
        else:
            b = m
    return m,it

#### Ejemplos

In [4]:
# Utilizando las funciones de prueba
x,i = biseccion(fun1,-5,5)
print(f'Un cero de la función f1 es {x:2.4e} ya que f({x:2.4e}) = {fun1(x):2.4e}')
print(f'El método necesito {i} iteraciones')
x,i = biseccion(fun2,-4,1)
print(f'\nUn cero de la función f2 es {x:2.4e} ya que f({x:2.4e}) = {fun2(x):2.4e}')
print(f'El método necesito {i} iteraciones')

Un cero de la función f1 es 1.0000e+00 ya que f(1.0000e+00) = 1.1176e-08
El método necesito 30 iteraciones

Un cero de la función f2 es -5.5879e-09 ya que f(-5.5879e-09) = 2.2352e-08
El método necesito 29 iteraciones


**Nota**: *Si la función evaluada en los extremos del intervalo de busqueda no tienen signos contrarios, el algoritmo puede no converger al cero a pesar de que se encuentre del intervalo*.

En el ejemplo anterior para la función $f(x) = x^{2} - 4\sin(x)$, si se toma el intervalo de búsqueda $[-4,2]$ el método no converge ya que $f(-4)f(2)>0$.

### Método de Newton

El *método de Newton* (conocido también como el método de *Newton-Raphson*) también es un algoritmo para buscar aproximaciones de los ceros o raíces de una función real. Este método incluso puede ser usado para buscar el máximo o mínimo de una función, al buscar los ceros de su primera derivada. 

Una forma de obtener el algoritmo es desarrollando la función $f(x)$ en series de Taylor, alrededor del punto $x_n$
$$
f(x) = f(x_{n}) + (x-x_{n})f'(x_{n}) + (x-x_{n})^{2}\frac{f''(x_{n})}{2!} + \cdots
$$
Si se trunca el desarrollo a partir del término de grado 2,  y evaluamos en $x_{n+1}$
$$
f(x_{n+1}) = f(x_{n})+f'(x_{n})(x_{n+1}-x_{n})
$$
Si además se supone que $x_{n+1}$ tiende a la raíz, se ha de cumplir que $f(x_{n+1}) = 0$, luego, sustituyendo en la expresión anterior, obtenemos el algoritmo al despejar $x_{n+1}$
$$
    x_{n+1} = x_{n} - \frac{f(x_{n})}{f'(x_{n})}
$$

Esta última ecuación nos da un algoritmo iterativo que genera una sucesión de puntos a partir de un valor inicial arbitrario $x_{0}$ que generalmente convergerá siempre que este estimado inicial esté lo suficientemente cerca del cero desconocido, y que $f'(x_{0})\neq0$.

#### Algoritmo (método de Newton)

* *Entrada* - $f$ función continua en $[a,b]$, $x_{0}$ aproximación inicial.
* *Salida*  - Raíz de $f$.

1. **Definir** $tol$.
2. **Definir** $n=0$.
3. **Mientras** $|x_{n}-x_{n+1}| > tol$ **hacer**
    1. **Si** $f'(x_{n}) = 0$ **entonces**
        1. **Detener**
    2. Calcular $$x_{n+1} = x_{n} - \frac{f(x_{n})}{f'(x_{n})}$$
    3. $n = n+1$
3. **Regresar** $x_{n}$.

In [5]:
def newton(f,x,tol = 1e-8, maxiter = 500):
    """Busca un cero de la funcion f mediante el metodo de Newton
    - Entrada >
      f (function)    - Funcion de la cual se quiere conocer su raiz.
      x (float)       - Punto inicial.
      tol (float)     - Opcional. Tolerancia para la diferencia entre dos aproximaciones.
      maxiter (float) - Opcional. Numero maximo de iteraciones.
    
    - Salida >
      cero (float) - Valor aproximado de un cero .
      it (int)     - Número de iteraciones que le toma al metodo encontrar un cero.
    """
    it,h = 0,1
    # Se calcula el cero mediante Newton
    while abs(h) > tol and it < maxiter:
        try:
            h = f(x)/derivada(f,x)
        except ZeroDivisionError:
            print(f'La derivada de la función en {x} es cero')
            return None
        cero = x - h
        x = cero
        it += 1
    return cero,it

#### Ejemplos

In [6]:
# Utilizando las funciones de prueba
x,i = newton(fun1,5)
print(f'Un cero de la función f1 es {x:2.4e} ya que f({x:2.4e}) = {fun1(x):2.4e}')
print(f'El método necesito {i} iteraciones')
x,i = newton(fun2,-0.5)
print(f'\nUn cero de la función f2 es {x:2.4e} ya que f({x:2.4e}) = {fun2(x):2.4e}')
print(f'El método necesito {i} iteraciones')

Un cero de la función f1 es 1.0000e+00 ya que f(1.0000e+00) = 0.0000e+00
El método necesito 7 iteraciones

Un cero de la función f2 es -1.0572e-18 ya que f(-1.0572e-18) = 4.2286e-18
El método necesito 4 iteraciones


In [7]:
# Tomando un punto inicial no adecuado.
r = newton(lambda x:x**2-1,0)
print(r)

La derivada de la función en 0 es cero
None


### Método de la secante

Dado que no siempre es fácil tener información de la derivada en el método de Newton, se puede usar una aproximación para esta. Usando diferencias progresivas 
$$
    f'(x) = \dfrac{f(x + h) - f(x)}{h}
$$
y tomando $h = x_{n-1} - x_{n}$ se tiene que 
$$
    f'(x_n) = \dfrac{f(x_{n-1}) - f(x_{n})}{x_{n-1} - x_{n}} = \dfrac{f(x_{n}) - f(x_{n-1})}{x_{n} - x_{n-1}}
$$

Luego, sustituyendo la última expresión en
$$
    x_{n+1} = x_{n} - \frac{f(x_{n})}{f'(x_{n})}
$$
se tiene que
$$
    x_{n+1} = x_{n} - f(x_{n})\frac{x_{n} - x_{n-1}}{f(x_{n}) - f(x_{n-1})}
$$

Al igual que para el método de Newton, esta última ecuación nos da un algoritmo iterativo que genera una sucesión de puntos a partir de dos valores iniciales arbitrarios $x_{0}$ y $x_{1}$.

#### Algoritmo (método de la secante)

* *Entrada* - $f$ función continua en $[a,b]$, $x_{0}$ y  $x_{1}$ aproximaciones iniciales.
* *Salida*  - Raíz de $f$.

1. **Definir** $tol$.
2. **Definir** $n=1$.
3. **Mientras** $|x_{n}-x_{n+1}| > tol$ **hacer**
    1. **Si** $f(x_{n}) - f(x_{n-1}) = 0$ **entonces**
        1. **Detener**
    2. Calcular $$x_{n+1} = x_{n} - f(x_{n})\frac{x_{n} - x_{n-1}}{f(x_{n}) - f(x_{n-1})}$$
    3. $n = n+1$
3. **Regresar** $x_{n+1}$.

In [8]:
def secante(f,x0,x1,tol = 1e-8, maxiter = 500):
    """Busca un cero de la funcion f mediante el metodo de la secante
    - Entrada >
      f (function)    - Funcion de la cual se quiere conocer su raiz.
      x0 (float)      - Primera aproximación.
      x1 (float)      - Segunda aproximación.
      tol (float)     - Opcional. Tolerancia para la diferencia entre dos aproximaciones.
      maxiter (float) - Opcional. Numero maximo de iteraciones.
    
    - Salida >
      cero (float) - Valor aproximado de un cero .
      it (int)     - Número de iteraciones que le toma al metodo encontrar un cero.
    """
    it = 0
    while abs(x0 - x1) > tol and it < maxiter:
        try:
            x0,x1 = x1,x0 - f(x0)*(x1-x0)/(f(x1)-f(x0))
        except ZeroDivisionError:
            print('La recta secante no tiene interseccion con el eje x')
            return None
        it += 1
    return x1,it

#### Ejemplos

In [9]:
# Utilizando las funciones de prueba
x,i = secante(fun1,-5,2)
print(f'Un cero de la función f1 es {x:2.4e} ya que f({x:2.4e}) = {fun1(x):2.4e}')
print(f'El método necesito {i} iteraciones')
x,i = secante(fun2,-4,2)
print(f'\nUn cero de la función f2 es {x:2.4e} ya que f({x:2.4e}) = {fun2(x):2.4e}')
print(f'El método necesito {i} iteraciones')

Un cero de la función f1 es 1.0000e+00 ya que f(1.0000e+00) = 0.0000e+00
El método necesito 9 iteraciones

Un cero de la función f2 es 1.9338e+00 ya que f(1.9338e+00) = 1.2434e-14
El método necesito 6 iteraciones


In [10]:
# Tomando puntos iniciales no adecuados.
r = secante(lambda x:x**2-1,-2,2)
print(r)

La recta secante no tiene interseccion con el eje x
None


### Función de prueba

La siguiente función muestra de manera interactiva el funcionamiento delos métodos para buscar los cero de una función. **Es importante haber ejecutado todas las celdas con las definiciones de las funciones y los módulos para que no haya errores**.

In [11]:
# Funciones auxiliares
## Controles
n_widget = widgets.IntSlider(value=50,min=3,max=300,description='N. nodos:',continuous_update=False)
fun_widget = widgets.Text(value='lambda x:x',description='Funcion:',continuous_update=False)
dom_widget = widgets.FloatRangeSlider(value = [-1.0, 1.0],min=-10,max=10,step=0.1,\
                               description='Intervalo:',continuous_update=False)
met__widget = widgets.Dropdown(options=[('Bisección', 'biseccion'), ('Newton', 'newton'), ('Secante', 'secante')],\
                               value='newton', description='Método:')
pini_widget = widgets.FloatSlider(value=0.5,min=-1,max=1,description='Punto inicial:',continuous_update=False)
tol_widget = widgets.FloatLogSlider(value=-6,base=10,min=-12,max=-1,step=0.1,description='Tolerancia:',\
                                 continuous_update=False)
miter_widget = widgets.IntSlider(value=500,min=10,max=500,description='N. max iteraciones:',continuous_update=False)

## Funcion para actualizar el rango del control del punto inicial
def update_pini_range(*args):
    """Modifica el rango para el control del punto inicial."""
    pini_widget.min = dom_widget.value[0]
    pini_widget.max = dom_widget.value[1]
dom_widget.observe(update_pini_range,'value')

In [12]:
# Funcion para buscar ceros
def buscaCero(n = 50,fun = lambda x:x,dom = [-1.0, 1.0],metodo = 'newton', pini = 0.5,tol = 1e-8,maxiter = 500):
    """Busca un cero mediante los métodos de bisección, newton y de la secante. Se muestra los resultados
    de manera gráfica.

    - Entrada >    
      n (int)          - Numero de puntos para graficar la funcion.
      fun (str)        - Expresion de la funcion de la cual se quiere encontrar un cero
      dom (list-float) - Dominio en 1D donde se busca el cero. También es el dominio de graficacion.
                         Para los métodos de bisección y secante estos valores se usan como los valores
                         iniciales.
      metodo (str)     - Indica el método a utilizar. Puede ser 'biseccion', 'newton' o 'secante'
      pini (float)     - Valor inicial sólo utilizado en el método de Newton
      tol (float)      - Tolerancia para la diferencia entre dos aproximaciones.
      maxiter (float)  - Numero maximo de iteraciones.
    """
    metDict = {'biseccion':biseccion,'newton':newton,'secante':secante}
    p_eval = np.linspace(dom[0],dom[1])
    f = eval(fun)
    if metodo.lower() == 'newton':
        r = metDict[metodo.lower()](f,pini,tol,maxiter)
    else:
        r = metDict[metodo.lower()](f,dom[0],dom[1],tol,maxiter)
    if r is not None:
        plt.plot(p_eval,np.zeros(p_eval.shape),'--',color = 'black',label='Eje x')
        plt.plot(p_eval,f(p_eval),color = 'steelblue',label='Funcion')        
        plt.scatter(r[0],f(r[0]),color = 'darkred',label='Cero')
        ax = plt.gca();
        ax.legend(loc = 'lower left')
        ax.set_xlabel("$x$")
        ax.set_title(f"$f({x:2.6e}) = {f(x):2.6e}$")
        plt.show()
        print(f'Un cero de la función dada es {x:2.4e} ya que f({x:2.4e}) = {fun1(x):2.4e}')
        print(f'El método {metodo} necesito {r[1]} iteraciones.')

interact(buscaCero,n = n_widget,fun = fun_widget,dom = dom_widget,metodo = met__widget,\
         pini = pini_widget,tol = tol_widget,maxiter = miter_widget);

interactive(children=(IntSlider(value=50, continuous_update=False, description='N. nodos:', max=300, min=3), T…